In [1]:
import re
import qiskit
import json
import time

from qiskit import transpile
import time
import os

from fast_generator import fc_tree_commute_recur_lookahead_fast

from benchmarks.UCCSD_entanglers import generate_UCCSD_entanglers
# from circuit_generator import construct_qcc_circuit

In [2]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False, threshold = 1):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                paulis = json.load(file)

            # Function to check if a string contains only 'I' characters
            def is_all_identity(pauli):
                return all(char == 'I' for char in pauli)

            # Filter the list to remove all identity Paulis
            test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
            test_params = [0.01 * i for i in range(len(test_paulis))]

            # Measure time for our method
            start_time = time.time()
            opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            end_time = time.time()
            our_time = end_time - start_time

            start_time = time.time()
            opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
            opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            end_time = time.time()
            combined_time = end_time - start_time
            
            # # Measure time for Qiskit method
            # start_time = time.time()
            # origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
            # origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
            # end_time = time.time()
            # qiskit_time = end_time - start_time
            #
            # # Measure time for RustiQ method
            # start_time = time.time()
            # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
            # end_time = time.time()
            # rustiq_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": len(test_paulis),
                "times": {
                    "our_time": our_time,
                    "combined_time": combined_time,
                    # "qiskit_time": qiskit_time,
                    # "rustiq_time": rustiq_time
                },
                "gate_counts": {
                    "our_method": opt_qc_f.count_ops().get('cx', 0),
                    "combined_method": opt_qiskit.count_ops().get('cx', 0),
                    # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
                    # "rustiq_method": entangling_count(circuit)
                },
                "circuit_entangling_depth": {
                    "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
                    "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
                    # "rustiq_method": entangling_depth(circuit)
                },
                "test_paulis_file": f'benchmarks/results/test_new_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_new_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [3]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, threshold = 1, filename = "Paulis"):

    results = []

    def is_all_identity(pauli):
        return all(char == 'I' for char in pauli)
    paulis = test_paulis
    # Filter the list to remove all identity Paulis
    test_paulis = [pauli for pauli in paulis if not is_all_identity(pauli)]
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]

    # Measure time for our method
    start_time = time.time()
    opt_qc_f, append_clifford_f, sorted_entanglers_f = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    end_time = time.time()
    our_time = end_time - start_time

    start_time = time.time()
    opt_qc_f2, append_clifford_f2, sorted_entanglers_f2 = fc_tree_commute_recur_lookahead_fast(entanglers=test_paulis, params=test_params, barrier=False, threshold=threshold)
    opt_qiskit = transpile(opt_qc_f2, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    end_time = time.time()
    combined_time = end_time - start_time
    
    # # Measure time for Qiskit method
    # start_time = time.time()
    # origin_qc = construct_qcc_circuit(entanglers=test_paulis, params=test_params, barrier=False)
    # origin_qiskit = transpile(origin_qc, optimization_level=3, basis_gates=["cx", "sx", "x", "rz"])
    # end_time = time.time()
    # qiskit_time = end_time - start_time

    # # Measure time for RustiQ method
    # start_time = time.time()
    # circuit = pauli_network_synthesis(test_paulis, Metric.COUNT, True, fix_clifford=True)
    # end_time = time.time()
    # rustiq_time = end_time - start_time
    
    # Collect results
    result = {
        "num_paulis": len(test_paulis),
        "times": {
            "our_time": our_time,
            "combined_time": combined_time,
            # "qiskit_time": qiskit_time,
            # "rustiq_time": rustiq_time
        },
        "gate_counts": {
            "our_method": opt_qc_f.count_ops().get('cx', 0),
            "combined_method": opt_qiskit.count_ops().get('cx', 0),
            # "qiskit_method": origin_qiskit.count_ops().get('cx', 0),
            # "rustiq_method": entangling_count(circuit)
        },
        "circuit_entangling_depth": {
            "our_method": opt_qc_f.depth(lambda instr: len(instr.qubits) > 1),
            "combined_method": opt_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "qiskit_method": origin_qiskit.depth(lambda instr: len(instr.qubits) > 1),
            # "rustiq_method": entangling_depth(circuit)
        },
        "test_paulis_file": f'benchmarks/results/test_new_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_new_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return sorted_entanglers_f
    

    

In [24]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20,]# 24]
#First evaluate the UCCSD ansatz:
for i, (x, y) in enumerate(zip(electrons_list, orbitals_list)):
    test_paulis = generate_UCCSD_entanglers(x, y)
    entanglers = run_experiment_paulis(test_paulis = test_paulis, save_output = True, threshold = 1, filename=f"Paulis{len(test_paulis)}.json")

{'num_paulis': 24, 'times': {'our_time': 0.09674191474914551, 'combined_time': 0.132676362991333}, 'gate_counts': {'our_method': 27, 'combined_method': 23}, 'circuit_entangling_depth': {'our_method': 21, 'combined_method': 17}, 'test_paulis_file': 'benchmarks/results/test_new_Paulis24.json'}
{'num_paulis': 80, 'times': {'our_time': 0.39171433448791504, 'combined_time': 0.5610082149505615}, 'gate_counts': {'our_method': 119, 'combined_method': 106}, 'circuit_entangling_depth': {'our_method': 94, 'combined_method': 82}, 'test_paulis_file': 'benchmarks/results/test_new_Paulis80.json'}
{'num_paulis': 320, 'times': {'our_time': 2.171626091003418, 'combined_time': 3.0018110275268555}, 'gate_counts': {'our_method': 507, 'combined_method': 448}, 'circuit_entangling_depth': {'our_method': 385, 'combined_method': 335}, 'test_paulis_file': 'benchmarks/results/test_new_Paulis320.json'}
{'num_paulis': 1656, 'times': {'our_time': 21.152221202850342, 'combined_time': 21.65596342086792}, 'gate_counts'

In [38]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True, threshold = 1)

benzene.json
{'num_paulis': 1254, 'times': {'our_time': 17.53726077079773, 'combined_time': 18.447914361953735}, 'gate_counts': {'our_method': 2479, 'combined_method': 2470}, 'circuit_entangling_depth': {'our_method': 1486, 'combined_method': 1481}, 'test_paulis_file': 'benchmarks/results/test_new_benzene.json'}
H2O.json
{'num_paulis': 184, 'times': {'our_time': 1.3753526210784912, 'combined_time': 1.7921802997589111}, 'gate_counts': {'our_method': 277, 'combined_method': 274}, 'circuit_entangling_depth': {'our_method': 191, 'combined_method': 189}, 'test_paulis_file': 'benchmarks/results/test_new_H2O.json'}
LiH.json
{'num_paulis': 61, 'times': {'our_time': 0.35505127906799316, 'combined_time': 0.399932861328125}, 'gate_counts': {'our_method': 76, 'combined_method': 74}, 'circuit_entangling_depth': {'our_method': 62, 'combined_method': 60}, 'test_paulis_file': 'benchmarks/results/test_new_LiH.json'}


In [9]:
#Then compare the MAXCUT paulis in max_cut_paulis folder

run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 1)

max_cut_benchmark_connected_n10_e12_l1.json
{'num_paulis': 22, 'times': {'our_time': 0.3031911849975586, 'combined_time': 0.32364630699157715}, 'gate_counts': {'our_method': 26, 'combined_method': 26}, 'circuit_entangling_depth': {'our_method': 21, 'combined_method': 21}, 'test_paulis_file': 'benchmarks/results/test_new_max_cut_benchmark_connected_n10_e12_l1.json'}
max_cut_benchmark_connected_n10_e12_l3.json
{'num_paulis': 66, 'times': {'our_time': 0.7307820320129395, 'combined_time': 0.8901388645172119}, 'gate_counts': {'our_method': 68, 'combined_method': 68}, 'circuit_entangling_depth': {'our_method': 47, 'combined_method': 47}, 'test_paulis_file': 'benchmarks/results/test_new_max_cut_benchmark_connected_n10_e12_l3.json'}
max_cut_benchmark_connected_n15_e63_l1.json
{'num_paulis': 78, 'times': {'our_time': 4.807599067687988, 'combined_time': 3.934091567993164}, 'gate_counts': {'our_method': 80, 'combined_method': 80}, 'circuit_entangling_depth': {'our_method': 47, 'combined_method': 

In [5]:
#since in the maxcut problems there are many commuting terms and it's slow to search through all of them,
#we can set the threshold to 2 to early stop the search once found a simplified pauli of weight 2
#you can see the run time difference, for the second benchmark runtime reduces from 30second to 7 seconds.

# run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", save_output = True, threshold = 2)
all_filenames=["max_cut_benchmark_regular_n20_e4_l1.json", "max_cut_benchmark_regular_n20_e4_l3.json", 
               "max_cut_benchmark_regular_n20_e8_l1.json", "max_cut_benchmark_regular_n20_e8_l3.json",
               "max_cut_benchmark_regular_n20_e12_l1.json", "max_cut_benchmark_regular_n20_e12_l3.json"]
for filename in all_filenames:
    run_experiment_folder(folder_path = "benchmarks/max_cut_paulis", filename=filename, save_output = True, threshold = 2)

max_cut_benchmark_regular_n20_e4_l1.json
{'num_paulis': 60, 'times': {'our_time': 2.371663808822632, 'combined_time': 2.1981241703033447}, 'gate_counts': {'our_method': 90, 'combined_method': 88}, 'circuit_entangling_depth': {'our_method': 35, 'combined_method': 34}, 'test_paulis_file': 'benchmarks/results/test_new_max_cut_benchmark_regular_n20_e4_l1.json'}
max_cut_benchmark_regular_n20_e4_l3.json
{'num_paulis': 180, 'times': {'our_time': 6.399900436401367, 'combined_time': 6.542515277862549}, 'gate_counts': {'our_method': 247, 'combined_method': 245}, 'circuit_entangling_depth': {'our_method': 99, 'combined_method': 98}, 'test_paulis_file': 'benchmarks/results/test_new_max_cut_benchmark_regular_n20_e4_l3.json'}
max_cut_benchmark_regular_n20_e8_l1.json
{'num_paulis': 100, 'times': {'our_time': 5.292855978012085, 'combined_time': 5.3028295040130615}, 'gate_counts': {'our_method': 129, 'combined_method': 129}, 'circuit_entangling_depth': {'our_method': 59, 'combined_method': 59}, 'test_p

In [11]:
#Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis", save_output = True, threshold= 2)

labs_n10_layers1.json
{'num_paulis': 80, 'times': {'our_time': 1.540156364440918, 'combined_time': 1.6080365180969238}, 'gate_counts': {'our_method': 106, 'combined_method': 106}, 'circuit_entangling_depth': {'our_method': 76, 'combined_method': 76}, 'test_paulis_file': 'benchmarks/results/test_new_labs_n10_layers1.json'}
labs_n10_layers3.json
{'num_paulis': 240, 'times': {'our_time': 4.689804315567017, 'combined_time': 4.497709512710571}, 'gate_counts': {'our_method': 296, 'combined_method': 296}, 'circuit_entangling_depth': {'our_method': 202, 'combined_method': 202}, 'test_paulis_file': 'benchmarks/results/test_new_labs_n10_layers3.json'}
labs_n11_layers1.json
{'num_paulis': 106, 'times': {'our_time': 3.075732946395874, 'combined_time': 3.61470103263855}, 'gate_counts': {'our_method': 137, 'combined_method': 137}, 'circuit_entangling_depth': {'our_method': 89, 'combined_method': 89}, 'test_paulis_file': 'benchmarks/results/test_new_labs_n11_layers1.json'}
labs_n11_layers3.json
{'num